In [3]:
# Import packages for memory monitoring
import os
import gc
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import pyproj
from rasterio.mask import mask
import matplotlib.pyplot as plt
from rasterstats import zonal_stats
import glob
from tqdm import tqdm
import warnings
import psutil  # For memory monitoring

# Suppress common warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Memory monitor function
def get_memory_usage():
    """Return the current memory usage in MB"""
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss / 1024 / 1024  # Convert to MB

# Configuration parameters
MAX_SAMPLE_SIZE = 10000  # Maximum number of pixels to sample per shapefile and raster
MEMORY_THRESHOLD_MB = 1000  # Trigger memory optimization if usage exceeds this value
CHUNK_SIZE = 1024  # Size of chunks when reading large rasters

# Set folders
data_folder = "G:/classification"  # Replace with your data folder
shapefile_folder = os.path.join(data_folder, "shapefiles")
raster_folder = os.path.join(data_folder, "rasters")
output_folder = os.path.join(data_folder, "results")

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Define shapefiles and raster patterns
shapefiles = {
    "baresoil": os.path.join(shapefile_folder, "Baresoil.shp"),
    "early_regrowth": os.path.join(shapefile_folder, "Early_regrowth.shp"),
    "mature_regrowth": os.path.join(shapefile_folder, "Mature_regrwoth.shp")
}

# Find all raster files - using CHM pattern based on error message
raster_files = glob.glob(os.path.join(raster_folder, "CHM_uav*.tif"))
print(f"Found {len(raster_files)} raster files")

# Create a log file for errors
log_file = os.path.join(output_folder, "processing_log.txt")
with open(log_file, 'w') as f:
    f.write("Canopy Height Processing Log\n")
    f.write("--------------------------\n")

def log_message(message):
    """Write message to log file and print to console"""
    print(message)
    with open(log_file, 'a') as f:
        f.write(message + "\n")

# Function to verify and fix CRS issues
def fix_crs_issues(gdf, raster_path):
    """Try multiple approaches to fix CRS issues between shapefile and raster"""
    try:
        # Handle missing CRS in shapefile
        if gdf.crs is None:
            log_message(f"Shapefile has no CRS defined. Setting to EPSG:4326 (WGS84)")
            gdf = gdf.set_crs(epsg=4326)
        
        # Check if raster path contains "S-JTSK" or "Krovak" in its CRS
        # This is a special case for Czech/Slovak S-JTSK Krovak projection
        if "CHM_uav" in raster_path:
            # Based on the error messages, we're dealing with EPSG:5514 (S-JTSK / Krovak East North)
            log_message(f"Detected S-JTSK Krovak CRS. Manually setting target CRS to EPSG:5514")
            
            # Instead of reading the raster CRS (which seems to be problematic),
            # manually define the S-JTSK / Krovak East North projection using EPSG code
            try:
                # First try with EPSG code (most reliable if supported)
                target_crs = "EPSG:5514"
                gdf_projected = gdf.to_crs(target_crs)
                log_message("Successfully reprojected to S-JTSK using EPSG:5514")
                return gdf_projected, None
            except Exception as e1:
                log_message(f"EPSG:5514 reprojection failed: {str(e1)}")
                
                # Try using a detailed WKT string for S-JTSK
                try:
                    # Standard WKT definition for S-JTSK / Krovak East North
                    s_jtsk_wkt = """
                    PROJCS["S-JTSK / Krovak East North",
                        GEOGCS["S-JTSK",
                            DATUM["System_of_the_Unified_Trigonometrical_Cadastral_Network",
                                SPHEROID["Bessel 1841",6377397.155,299.1528128,
                                    AUTHORITY["EPSG","7004"]],
                                TOWGS84[589,76,480,0,0,0,0],
                                AUTHORITY["EPSG","6156"]],
                            PRIMEM["Greenwich",0,
                                AUTHORITY["EPSG","8901"]],
                            UNIT["degree",0.0174532925199433,
                                AUTHORITY["EPSG","9122"]],
                            AUTHORITY["EPSG","4156"]],
                        PROJECTION["Krovak"],
                        PARAMETER["latitude_of_center",49.5],
                        PARAMETER["longitude_of_center",24.83333333333333],
                        PARAMETER["azimuth",30.28813972222222],
                        PARAMETER["pseudo_standard_parallel_1",78.5],
                        PARAMETER["scale_factor",0.9999],
                        PARAMETER["false_easting",0],
                        PARAMETER["false_northing",0],
                        UNIT["metre",1,
                            AUTHORITY["EPSG","9001"]],
                        AXIS["X",EAST],
                        AXIS["Y",NORTH],
                        AUTHORITY["EPSG","5514"]]
                    """
                    gdf_projected = gdf.to_crs(s_jtsk_wkt)
                    log_message("Successfully reprojected to S-JTSK using WKT definition")
                    return gdf_projected, None
                except Exception as e2:
                    log_message(f"WKT reprojection failed: {str(e2)}")
                    
                    # As a last resort, try with explicit coordinate operation
                    try:
                        log_message("Attempting direct coordinate transformation...")
                        # Create custom transformation pipeline
                        transformer = pyproj.Transformer.from_pipeline(
                            "+proj=pipeline +step +proj=unitconvert +xy_in=deg +xy_out=rad "
                            "+step +proj=krovak +lat_0=49.5 +lon_0=24.83333333333333 "
                            "+alpha=30.28813972222222 +k=0.9999 +x_0=0 +y_0=0"
                        )
                        
                        # Manual transformation
                        transformed_geometries = []
                        for geom in gdf.geometry:
                            if geom.geom_type == 'Polygon':
                                # Transform each point in the polygon
                                exterior_coords = []
                                for x, y in geom.exterior.coords:
                                    e, n = transformer.transform(y, x)  # Note: y, x order for lon, lat
                                    exterior_coords.append((e, n))
                                
                                # Create new polygon with transformed coordinates
                                transformed_geometries.append(
                                    gpd.GeoSeries([gpd.geom.Polygon(exterior_coords)], crs="EPSG:5514")[0]
                                )
                            else:
                                # Skip non-polygon geometries
                                log_message(f"Skipping non-polygon geometry: {geom.geom_type}")
                        
                        # Create new GeoDataFrame with transformed geometries
                        gdf_projected = gpd.GeoDataFrame(
                            gdf.drop(columns='geometry'),
                            geometry=transformed_geometries,
                            crs="EPSG:5514"
                        )
                        
                        log_message("Successfully completed manual point-by-point transformation")
                        return gdf_projected, None
                    except Exception as e3:
                        log_message(f"Manual transformation failed: {str(e3)}")
                        
                        # Final fallback: use alternative raster-based approach
                        log_message("All reprojection methods failed. Will attempt direct pixel extraction.")
                        return None, "All reprojection methods failed"
        else:
            # For non-S-JTSK rasters, use the standard approach
            with rasterio.open(raster_path) as src:
                raster_crs = src.crs
            
            # Handle missing CRS in raster
            if raster_crs is None:
                log_message(f"Warning: Raster {os.path.basename(raster_path)} has no CRS defined.")
                return None, "Raster has no CRS"
            
            # Try direct reprojection if CRS differ
            if gdf.crs != raster_crs:
                try:
                    log_message(f"Reprojecting from {gdf.crs} to {raster_crs}")
                    gdf_projected = gdf.to_crs(raster_crs)
                    return gdf_projected, None
                except Exception as e1:
                    log_message(f"Direct reprojection failed: {str(e1)}")
                    
                    # Try with explicit PROJ4 string if available
                    try:
                        log_message("Trying with explicit PROJ4 string...")
                        proj4 = pyproj.CRS(raster_crs).to_proj4()
                        gdf_projected = gdf.to_crs(proj4)
                        return gdf_projected, None
                    except Exception as e2:
                        log_message(f"PROJ4 approach failed: {str(e2)}")
                        
                        # Try two-step reprojection through WGS84
                        try:
                            log_message("Attempting two-step reprojection via EPSG:4326...")
                            gdf_wgs84 = gdf.to_crs(epsg=4326)
                            gdf_projected = gdf_wgs84.to_crs(raster_crs)
                            return gdf_projected, None
                        except Exception as e3:
                            log_message(f"Two-step reprojection failed: {str(e3)}")
                            return None, "All reprojection methods failed"
            else:
                # CRS already match
                return gdf, None
    except Exception as e:
        log_message(f"CRS processing error: {str(e)}")
        return None, f"CRS processing error: {str(e)}"

# Function to extract pixel values from raster for a given shapefile
def extract_values(shapefile_path, raster_path):
    """Extract values from raster that fall within the shapefile polygons using sampling"""
    initial_memory = get_memory_usage()
    log_message(f"Starting extraction. Memory usage: {initial_memory:.1f} MB")
    
    try:
        # Read shapefile
        gdf = gpd.read_file(shapefile_path)
        log_message(f"Processing {os.path.basename(shapefile_path)} with {os.path.basename(raster_path)}")
        
        # Fix CRS issues
        gdf_projected, error = fix_crs_issues(gdf, raster_path)
        
        # If reprojection failed but it's S-JTSK, try alternative approach
        if error and "CHM_uav" in raster_path:
            log_message("Attempting alternative extraction method for S-JTSK data...")
            
            # Since we couldn't reproject, we'll try to extract values using the bounds
            # of the shapefile and then filter post-extraction
            try:
                # Load the raster
                with rasterio.open(raster_path) as src:
                    # Check for NoData value
                    nodata = src.nodata
                    log_message(f"Raster NoData value: {nodata}")
                    
                    # Read a sample of the raster instead of the whole thing
                    # Define a window size based on raster dimensions
                    height, width = src.shape
                    window_height = min(CHUNK_SIZE, height)
                    window_width = min(CHUNK_SIZE, width)
                    
                    # Calculate number of windows needed
                    num_windows_y = (height + window_height - 1) // window_height
                    num_windows_x = (width + window_width - 1) // window_width
                    
                    total_values = []
                    
                    # Track how many samples we've collected
                    sample_count = 0
                    
                    log_message(f"Sampling raster in {num_windows_y}x{num_windows_x} windows...")
                    
                    # Process a subset of windows to save memory
                    # Select a portion of windows to sample randomly
                    max_windows = min(20, num_windows_y * num_windows_x)  # Limit total windows
                    sample_window_indices = np.random.choice(
                        num_windows_y * num_windows_x, 
                        size=max_windows, 
                        replace=False
                    )
                    
                    for window_idx in sample_window_indices:
                        if sample_count >= MAX_SAMPLE_SIZE:
                            break
                            
                        # Convert linear index to 2D coordinates
                        window_y = window_idx // num_windows_x
                        window_x = window_idx % num_windows_x
                        
                        # Calculate window bounds
                        row_start = window_y * window_height
                        col_start = window_x * window_width
                        row_end = min(row_start + window_height, height)
                        col_end = min(col_start + window_width, width)
                        
                        # Read window
                        window = src.read(1, window=((row_start, row_end), (col_start, col_end)))
                        
                        # Filter out NoData values
                        if nodata is not None:
                            valid_mask = window != nodata
                        else:
                            valid_mask = ~np.isnan(window)
                        
                        valid_values = window[valid_mask]
                        
                        if len(valid_values) > 0:
                            # Sample a limited number from this window
                            if len(valid_values) > MAX_SAMPLE_SIZE // max_windows:
                                indices = np.random.choice(
                                    len(valid_values), 
                                    size=MAX_SAMPLE_SIZE // max_windows, 
                                    replace=False
                                )
                                valid_values = valid_values[indices]
                            
                            total_values.extend(valid_values.tolist())
                            sample_count += len(valid_values)
                    
                    # Force garbage collection to free memory
                    gc.collect()
                    
                    log_message(f"Extracted {len(total_values)} sampled values. Memory usage: {get_memory_usage():.1f} MB")
                    return total_values
                    
            except Exception as e:
                log_message(f"Alternative extraction method failed: {str(e)}")
                log_message("Unable to process this raster. Skipping.")
                return []
        
        # If standard reprojection failed and alternative approach isn't applicable
        if error and "CHM_uav" not in raster_path:
            log_message(f"CRS error for {os.path.basename(raster_path)}: {error}")
            return []
            
        # Continue with standard approach if reprojection succeeded
        # Use optimized zonal stats approach with sampling
        try:
            # First check memory usage
            current_memory = get_memory_usage()
            log_message(f"Current memory usage before zonal stats: {current_memory:.1f} MB")
            
            # Use small sample size for each polygon
            all_values = []
            
            # If we have many polygons, sample them
            if len(gdf_projected) > 5:
                # Sample a max of 5 polygons
                sample_size = min(5, len(gdf_projected))
                sample_indices = np.random.choice(len(gdf_projected), size=sample_size, replace=False)
                gdf_sample = gdf_projected.iloc[sample_indices].copy()
                log_message(f"Sampling {sample_size} polygons from {len(gdf_projected)} total")
            else:
                gdf_sample = gdf_projected
            
            # Process each polygon separately to control memory usage
            for idx, row in gdf_sample.iterrows():
                # Create a GeoDataFrame with just this polygon
                single_polygon_gdf = gpd.GeoDataFrame(geometry=[row.geometry], crs=gdf_projected.crs)
                
                # Use zonal_stats with raster_out to get pixel values
                stats = zonal_stats(single_polygon_gdf, raster_path, 
                                   stats=['count'], 
                                   nodata=np.nan,
                                   all_touched=False,
                                   geojson_out=True,
                                   raster_out=True)
                
                # Extract values from the mini raster
                for feature in stats:
                    if 'properties' in feature and 'mini_raster_array' in feature['properties']:
                        mini_raster = feature['properties']['mini_raster_array']
                        
                        # Flatten and filter out NoData values
                        values = mini_raster.flatten()
                        valid_values = values[~np.isnan(values)]
                        
                        # Sample values if there are too many
                        if len(valid_values) > MAX_SAMPLE_SIZE // len(gdf_sample):
                            sample_indices = np.random.choice(
                                len(valid_values),
                                size=MAX_SAMPLE_SIZE // len(gdf_sample),
                                replace=False
                            )
                            valid_values = valid_values[sample_indices]
                        
                        all_values.extend(valid_values.tolist())
                        log_message(f"  Extracted {len(valid_values)} values from polygon {idx}")
                
                # Force garbage collection after each polygon
                gc.collect()
            
            if all_values:
                log_message(f"Successfully extracted {len(all_values)} sampled values. Memory: {get_memory_usage():.1f} MB")
                return all_values
            else:
                log_message("No values extracted with zonal_stats, trying mask method...")
        except Exception as e:
            log_message(f"Zonal stats method failed: {str(e)}")
            log_message("Falling back to mask method...")
        
        # Fallback to mask method with sampling
        log_message("Using mask method with sampling...")
        all_values = []
        
        try:
            with rasterio.open(raster_path) as src:
                # Sample polygons if there are many
                if len(gdf_projected) > 5:
                    sample_size = min(5, len(gdf_projected))
                    sample_indices = np.random.choice(len(gdf_projected), size=sample_size, replace=False)
                    gdf_sample = gdf_projected.iloc[sample_indices].copy()
                else:
                    gdf_sample = gdf_projected
                
                # Process each polygon
                for idx, geom in enumerate(gdf_sample.geometry):
                    try:
                        # Get pixel values within the geometry
                        out_image, out_transform = mask(src, [geom.__geo_interface__], crop=True, nodata=np.nan)
                        
                        # Flatten and filter out NoData values
                        valid_pixels = out_image[0].flatten()
                        valid_pixels = valid_pixels[~np.isnan(valid_pixels)]
                        
                        # Sample if there are too many values
                        if len(valid_pixels) > MAX_SAMPLE_SIZE // len(gdf_sample):
                            sample_indices = np.random.choice(
                                len(valid_pixels),
                                size=MAX_SAMPLE_SIZE // len(gdf_sample),
                                replace=False
                            )
                            valid_pixels = valid_pixels[sample_indices]
                        
                        all_values.extend(valid_pixels.tolist())
                        log_message(f"  Extracted {len(valid_pixels)} values from geometry {idx+1}/{len(gdf_sample)}")
                        
                        # Check memory usage and clean up if necessary
                        if get_memory_usage() > MEMORY_THRESHOLD_MB:
                            log_message(f"Memory threshold exceeded. Cleaning up. Memory: {get_memory_usage():.1f} MB")
                            gc.collect()
                            
                    except Exception as e:
                        log_message(f"Error masking geometry {idx+1}: {str(e)}")
                        continue
        except Exception as e:
            log_message(f"Mask method failed: {str(e)}")
            
        final_memory = get_memory_usage()
        log_message(f"Total values extracted: {len(all_values)}. Final memory: {final_memory:.1f} MB")
        return all_values
    
    except Exception as e:
        log_message(f"Error processing {os.path.basename(shapefile_path)} with {os.path.basename(raster_path)}: {str(e)}")
        # Force cleanup
        gc.collect()
        return []

# Dictionary to store all values by land cover type
all_values_by_type = {
    "baresoil": [],
    "early_regrowth": [],
    "mature_regrowth": []
}

# Set raster paths that should be skipped (if any)
skip_rasters = []

# Process each shapefile with each raster
for land_type, shapefile_path in shapefiles.items():
    log_message(f"\nProcessing {land_type}...")
    
    # Skip if shapefile doesn't exist
    if not os.path.exists(shapefile_path):
        log_message(f"Warning: {shapefile_path} does not exist. Skipping.")
        continue
    
    # Process each raster file
    for raster_path in tqdm(raster_files, desc=f"Rasters for {land_type}"):
        # Skip problematic rasters if needed
        if os.path.basename(raster_path) in skip_rasters:
            log_message(f"Skipping known problematic raster: {os.path.basename(raster_path)}")
            continue
            
        values = extract_values(shapefile_path, raster_path)
        all_values_by_type[land_type].extend(values)
    
    log_message(f"Collected {len(all_values_by_type[land_type])} pixel values for {land_type}")

# Check if we have values to process
any_values = any(len(values) > 0 for values in all_values_by_type.values())
if not any_values:
    log_message("No valid values were extracted. Check shapefile and raster compatibility.")
    exit(1)

# Create histograms
plt.figure(figsize=(15, 10))

# Set colors
colors = {
    "baresoil": '#D55E00',      # Orange
    "early_regrowth": '#0072B2', # Blue
    "mature_regrowth": '#009E73' # Green
}

# Plot histograms for each land cover type
subplot_index = 1
has_valid_data = False

for land_type, values in all_values_by_type.items():
    # Skip if no values
    if not values or len(values) == 0:
        log_message(f"No values for {land_type}, skipping histogram")
        continue
        
    # Convert to numpy array
    values_array = np.array(values)
    
    # Check for valid values
    valid_mask = ~np.isnan(values_array)
    if not np.any(valid_mask):
        log_message(f"No valid non-NaN values for {land_type}, skipping")
        continue
        
    # Filter to only valid values
    values_array = values_array[valid_mask]
    
    # Double-check we have data
    if len(values_array) == 0:
        log_message(f"No valid values for {land_type} after filtering NaNs")
        continue
        
    has_valid_data = True
    
    # Filter out extreme values for better visualization
    if len(values_array) > 10:  # Only filter if we have enough values
        try:
            # Remove outliers (values beyond 3 standard deviations)
            mean = np.mean(values_array)
            std = np.std(values_array)
            
            if std > 0:  # Only filter if standard deviation is positive
                filtered_values = values_array[(values_array > mean - 3 * std) & (values_array < mean + 3 * std)]
                
                # If filtering removed too many values, use original
                if len(filtered_values) < 0.5 * len(values_array):
                    log_message(f"Warning: Outlier filtering removed too many values for {land_type}. Using original values.")
                    filtered_values = values_array
            else:
                log_message(f"Standard deviation is 0 for {land_type}, skipping outlier filtering")
                filtered_values = values_array
                
        except Exception as e:
            log_message(f"Error in outlier filtering for {land_type}: {str(e)}")
            filtered_values = values_array
    else:
        filtered_values = values_array
    
    # Create histogram
    plt.subplot(3, 1, subplot_index)
    subplot_index += 1
    
    # Verify we have valid data to plot
    if len(filtered_values) > 0:
        # Plot histogram with density=True for probability density
        try:
            hist, bins, _ = plt.hist(filtered_values, bins=min(50, len(np.unique(filtered_values))), 
                                    alpha=0.7, color=colors[land_type], density=True,
                                    label=f"n={len(filtered_values)}")
            
            # Add vertical line for mean
            plt.axvline(np.mean(filtered_values), color='black', linestyle='dashed', linewidth=1, 
                       label=f'Mean: {np.mean(filtered_values):.2f}m')
        except Exception as e:
            log_message(f"Error creating histogram for {land_type}: {str(e)}")
            # Fallback to basic plot if histogram fails
            plt.plot([0], [0], color=colors[land_type], label=f"Error: Could not plot {land_type}")
    else:
        # No data placeholder
        plt.text(0.5, 0.5, f"No valid data for {land_type}", 
                horizontalalignment='center', verticalalignment='center',
                transform=plt.gca().transAxes, fontsize=14)
    
    # Add title and labels
    pretty_name = land_type.replace('_', ' ').title()
    plt.title(f'{pretty_name} - Canopy Height Distribution', fontsize=14)
    plt.xlabel('Canopy Height (m)', fontsize=12)
    plt.ylabel('Density', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.legend()

# Adjust layout only if we have data
if has_valid_data:
    plt.tight_layout()
    
    # Save combined figure
    combined_plot_path = os.path.join(output_folder, 'canopy_height_histograms.png')
    plt.savefig(combined_plot_path, dpi=300)
    plt.savefig(os.path.join(output_folder, 'canopy_height_histograms.pdf'))
    log_message(f"Combined histogram saved to {combined_plot_path}")
else:
    log_message("No valid data to plot histograms")
    plt.close()  # Close the empty figure

# Save individual histograms and statistics
for land_type, values in all_values_by_type.items():
    if not values or len(values) == 0:
        log_message(f"No data for {land_type}, skipping individual histogram")
        continue
        
    # Convert to array and filter non-finite values
    values_array = np.array(values)
    valid_mask = ~np.isnan(values_array) & np.isfinite(values_array)
    
    # Check if we have valid data
    if not np.any(valid_mask):
        log_message(f"No valid data points for {land_type}")
        continue
        
    # Keep only valid values
    values_array = values_array[valid_mask]
    
    # Double-check we have data
    if len(values_array) == 0:
        log_message(f"No valid values for {land_type} after filtering")
        continue
    
    # Only filter if we have enough values
    try:
        if len(values_array) > 10:
            # Check if standard deviation is positive
            if np.std(values_array) > 0:
                filtered_values = values_array[(values_array > np.mean(values_array) - 3 * np.std(values_array)) & 
                                           (values_array < np.mean(values_array) + 3 * np.std(values_array))]
                
                # If filtering removed too many values, use original
                if len(filtered_values) < 0.5 * len(values_array):
                    filtered_values = values_array
            else:
                filtered_values = values_array
        else:
            filtered_values = values_array
    except Exception as e:
        log_message(f"Error filtering outliers for {land_type}: {str(e)}")
        filtered_values = values_array
    
    # Create separate figure for each land type
    try:
        plt.figure(figsize=(10, 6))
        
        if len(filtered_values) > 0:
            hist, bins, _ = plt.hist(filtered_values, bins=min(50, len(np.unique(filtered_values))), 
                                     alpha=0.7, color=colors[land_type], density=True)
            
            mean_val = np.mean(filtered_values)
            plt.axvline(mean_val, color='black', linestyle='dashed', linewidth=1, 
                       label=f'Mean: {mean_val:.2f}m')
            
            pretty_name = land_type.replace('_', ' ').title()
            plt.title(f'{pretty_name} - Canopy Height Distribution', fontsize=14)
            plt.xlabel('Canopy Height (m)', fontsize=12)
            plt.ylabel('Density', fontsize=12)
            plt.grid(True, alpha=0.3)
            plt.legend()
            
            # Save individual plot
            plt.tight_layout()
            individual_plot_path = os.path.join(output_folder, f'{land_type}_histogram.png')
            plt.savefig(individual_plot_path, dpi=300)
            log_message(f"Individual histogram for {land_type} saved to {individual_plot_path}")
            plt.close()
            
            # Calculate statistics with error handling
            try:
                stats = {
                    'mean': np.mean(filtered_values),
                    'median': np.median(filtered_values),
                    'std_dev': np.std(filtered_values),
                    'min': np.min(filtered_values),
                    'max': np.max(filtered_values),
                    'count': len(filtered_values),
                    '25th_percentile': np.percentile(filtered_values, 25),
                    '75th_percentile': np.percentile(filtered_values, 75)
                }
                
                # Save statistics to CSV
                stats_df = pd.DataFrame([stats])
                stats_csv_path = os.path.join(output_folder, f'{land_type}_statistics.csv')
                stats_df.to_csv(stats_csv_path, index=False)
                log_message(f"Statistics for {land_type} saved to {stats_csv_path}")
            except Exception as e:
                log_message(f"Error calculating statistics for {land_type}: {str(e)}")
        else:
            log_message(f"No valid data to create individual histogram for {land_type}")
            plt.close()
    except Exception as e:
        log_message(f"Error creating individual histogram for {land_type}: {str(e)}")
        plt.close()

# Create a summary table with statistics for all land cover types
summary_rows = []
for land_type, values in all_values_by_type.items():
    if not values:
        continue
        
    values_array = np.array(values)
    if len(values_array) > 0:
        summary_rows.append({
            'Land_Cover': land_type.replace('_', ' ').title(),
            'Count': len(values_array),
            'Mean_Height_m': np.mean(values_array),
            'Median_Height_m': np.median(values_array),
            'Std_Dev_m': np.std(values_array),
            'Min_Height_m': np.min(values_array),
            'Max_Height_m': np.max(values_array),
            'Range_m': np.max(values_array) - np.min(values_array)
        })

if summary_rows:
    summary_df = pd.DataFrame(summary_rows)
    summary_csv_path = os.path.join(output_folder, 'canopy_height_summary.csv')
    summary_df.to_csv(summary_csv_path, index=False)
    log_message(f"Summary statistics saved to {summary_csv_path}")

    # Also create an Excel file with formatting
    summary_excel_path = os.path.join(output_folder, 'canopy_height_summary.xlsx')
    with pd.ExcelWriter(summary_excel_path) as writer:
        summary_df.to_excel(writer, sheet_name='Summary', index=False)
    log_message(f"Summary Excel file saved to {summary_excel_path}")

log_message("\nAnalysis complete! Results saved to: " + output_folder)

Found 43 raster files

Processing baresoil...


Rasters for baresoil:   0%|                                                                     | 0/43 [00:00<?, ?it/s]

Starting extraction. Memory usage: 215.9 MB
Processing Baresoil.shp with CHM_uav47.tif
Detected S-JTSK Krovak CRS. Manually setting target CRS to EPSG:5514
Successfully reprojected to S-JTSK using EPSG:5514
Current memory usage before zonal stats: 216.0 MB
Sampling 5 polygons from 117 total
  Extracted 2000 values from polygon 52
  Extracted 2000 values from polygon 24
  Extracted 2000 values from polygon 47
  Extracted 2000 values from polygon 68


Rasters for baresoil:   2%|█▍                                                           | 1/43 [00:00<00:10,  3.92it/s]

  Extracted 2000 values from polygon 79
Successfully extracted 10000 sampled values. Memory: 226.1 MB
Starting extraction. Memory usage: 214.5 MB
Processing Baresoil.shp with CHM_uav1.tif
Detected S-JTSK Krovak CRS. Manually setting target CRS to EPSG:5514
Successfully reprojected to S-JTSK using EPSG:5514
Current memory usage before zonal stats: 214.5 MB
Sampling 5 polygons from 117 total
  Extracted 2000 values from polygon 71
  Extracted 2000 values from polygon 78
  Extracted 2000 values from polygon 49
  Extracted 2000 values from polygon 113


...

  Extracted 2000 values from polygon 108
Successfully extracted 10000 sampled values. Memory: 251.1 MB
Starting extraction. Memory usage: 227.5 MB
Processing Early_regrowth.shp with CHM_uav46.tif
Detected S-JTSK Krovak CRS. Manually setting target CRS to EPSG:5514
Successfully reprojected to S-JTSK using EPSG:5514
Current memory usage before zonal stats: 227.6 MB
Sampling 5 polygons from 288 total
  Extracted 2000 values from polygon 252
  Extracted 2000 values from polygon 278
  Extracted 2000 values from polygon 186
  Extracted 2000 values from polygon 87
  Extracted 2000 values from polygon 169
Successfully extracted 10000 sampled values. Memory: 232.4 MB


Rasters for early_regrowth: 100%|██████████████████████████████████████████████████████| 43/43 [00:19<00:00,  2.23it/s]

Collected 428378 pixel values for early_regrowth

Processing mature_regrowth...


TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

<Figure size 1500x1000 with 0 Axes>

In [ ]:
"G:/classification"

In [4]:
# Import packages for memory monitoring
import os
import gc
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import pyproj
from rasterio.mask import mask
import matplotlib.pyplot as plt
from rasterstats import zonal_stats
import glob
from tqdm import tqdm
import warnings
import psutil  # For memory monitoring

# Suppress common warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Memory monitor function
def get_memory_usage():
    """Return the current memory usage in MB"""
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss / 1024 / 1024  # Convert to MB

# Configuration parameters
MAX_SAMPLE_SIZE = 100000  # Maximum number of pixels to sample per shapefile and raster
MEMORY_THRESHOLD_MB = 1000  # Trigger memory optimization if usage exceeds this value
CHUNK_SIZE = 520  # Size of chunks when reading large rasters

# Set folders
data_folder = "H:/classification"  # Replace with your actual folder path
shapefile_folder = os.path.join(data_folder, "shapefiles")
raster_folder = os.path.join(data_folder, "rasters")
output_folder = os.path.join(data_folder, "results")

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Define shapefiles and raster patterns
shapefiles = {
    "baresoil": os.path.join(shapefile_folder, "Baresoil.shp"),
    "early_regrowth": os.path.join(shapefile_folder, "Early_regrowth.shp"),
    "mature_regrowth": os.path.join(shapefile_folder, "Mature_regrowth2.shp")
}

# Find all raster files - using CHM pattern based on error message
raster_files = glob.glob(os.path.join(raster_folder, "CHM_uav*.tif"))
print(f"Found {len(raster_files)} raster files")

# Create a log file for errors
log_file = os.path.join(output_folder, "processing_log.txt")
with open(log_file, 'w') as f:
    f.write("Canopy Height Processing Log\n")
    f.write("--------------------------\n")

def log_message(message):
    """Write message to log file and print to console"""
    print(message)
    with open(log_file, 'a') as f:
        f.write(message + "\n")

# Function to clean and validate data for analysis
def clean_data_for_analysis(values):
    """Convert values to numeric and filter invalid entries"""
    if not values or len(values) == 0:
        return np.array([])
        
    # First, convert list to numpy array if it's not already
    if not isinstance(values, np.ndarray):
        values_array = np.array(values, dtype=object)
    else:
        values_array = values
    
    # Check if we need to convert from object type to numeric
    if values_array.dtype.kind == 'O':  # Object type
        log_message(f"Converting object array to numeric (sample: {values_array[:5]})")
        
        # Filter out None values first
        values_filtered = [v for v in values_array if v is not None]
        
        if len(values_filtered) == 0:
            log_message("All values were None, returning empty array")
            return np.array([])
            
        # Try to convert to float
        try:
            # Use pandas for better mixed-type conversion
            import pandas as pd
            numeric_values = pd.to_numeric(values_filtered, errors='coerce').dropna().values
            log_message(f"Converted {len(numeric_values)} values to numeric (out of {len(values_filtered)})")
            
            if len(numeric_values) == 0:
                log_message("No valid numeric values after conversion")
                return np.array([])
                
            return numeric_values
        except Exception as e:
            log_message(f"Error converting to numeric: {str(e)}")
            
            # Manual conversion as fallback
            numeric_values = []
            for v in values_filtered:
                try:
                    if v is not None:
                        numeric_values.append(float(v))
                except (ValueError, TypeError):
                    continue
                    
            log_message(f"Manual conversion yielded {len(numeric_values)} numeric values")
            return np.array(numeric_values)
    else:
        # Already numeric type, just filter NaN and inf values if float
        if values_array.dtype.kind in 'fc':  # float or complex
            mask = ~np.isnan(values_array) & np.isfinite(values_array)
            return values_array[mask]
        else:
            # For integers, just return as is
            return values_array

# Function to verify and fix CRS issues
def fix_crs_issues(gdf, raster_path):
    """Try multiple approaches to fix CRS issues between shapefile and raster"""
    try:
        # Handle missing CRS in shapefile
        if gdf.crs is None:
            log_message(f"Shapefile has no CRS defined. Setting to EPSG:4326 (WGS84)")
            gdf = gdf.set_crs(epsg=4326)
        
        # Check if raster path contains "S-JTSK" or "Krovak" in its CRS
        # This is a special case for Czech/Slovak S-JTSK Krovak projection
        if "CHM_uav" in raster_path:
            # Based on the error messages, we're dealing with EPSG:5514 (S-JTSK / Krovak East North)
            log_message(f"Detected S-JTSK Krovak CRS. Manually setting target CRS to EPSG:5514")
            
            # Instead of reading the raster CRS (which seems to be problematic),
            # manually define the S-JTSK / Krovak East North projection using EPSG code
            try:
                # First try with EPSG code (most reliable if supported)
                target_crs = "EPSG:5514"
                gdf_projected = gdf.to_crs(target_crs)
                log_message("Successfully reprojected to S-JTSK using EPSG:5514")
                return gdf_projected, None
            except Exception as e1:
                log_message(f"EPSG:5514 reprojection failed: {str(e1)}")
                
                # Try using a detailed WKT string for S-JTSK
                try:
                    # Standard WKT definition for S-JTSK / Krovak East North
                    s_jtsk_wkt = """
                    PROJCS["S-JTSK / Krovak East North",
                        GEOGCS["S-JTSK",
                            DATUM["System_of_the_Unified_Trigonometrical_Cadastral_Network",
                                SPHEROID["Bessel 1841",6377397.155,299.1528128,
                                    AUTHORITY["EPSG","7004"]],
                                TOWGS84[589,76,480,0,0,0,0],
                                AUTHORITY["EPSG","6156"]],
                            PRIMEM["Greenwich",0,
                                AUTHORITY["EPSG","8901"]],
                            UNIT["degree",0.0174532925199433,
                                AUTHORITY["EPSG","9122"]],
                            AUTHORITY["EPSG","4156"]],
                        PROJECTION["Krovak"],
                        PARAMETER["latitude_of_center",49.5],
                        PARAMETER["longitude_of_center",24.83333333333333],
                        PARAMETER["azimuth",30.28813972222222],
                        PARAMETER["pseudo_standard_parallel_1",78.5],
                        PARAMETER["scale_factor",0.9999],
                        PARAMETER["false_easting",0],
                        PARAMETER["false_northing",0],
                        UNIT["metre",1,
                            AUTHORITY["EPSG","9001"]],
                        AXIS["X",EAST],
                        AXIS["Y",NORTH],
                        AUTHORITY["EPSG","5514"]]
                    """
                    gdf_projected = gdf.to_crs(s_jtsk_wkt)
                    log_message("Successfully reprojected to S-JTSK using WKT definition")
                    return gdf_projected, None
                except Exception as e2:
                    log_message(f"WKT reprojection failed: {str(e2)}")
                    
                    # As a last resort, try with explicit coordinate operation
                    try:
                        log_message("Attempting direct coordinate transformation...")
                        # Create custom transformation pipeline
                        transformer = pyproj.Transformer.from_pipeline(
                            "+proj=pipeline +step +proj=unitconvert +xy_in=deg +xy_out=rad "
                            "+step +proj=krovak +lat_0=49.5 +lon_0=24.83333333333333 "
                            "+alpha=30.28813972222222 +k=0.9999 +x_0=0 +y_0=0"
                        )
                        
                        # Manual transformation
                        transformed_geometries = []
                        for geom in gdf.geometry:
                            if geom.geom_type == 'Polygon':
                                # Transform each point in the polygon
                                exterior_coords = []
                                for x, y in geom.exterior.coords:
                                    e, n = transformer.transform(y, x)  # Note: y, x order for lon, lat
                                    exterior_coords.append((e, n))
                                
                                # Create new polygon with transformed coordinates
                                transformed_geometries.append(
                                    gpd.GeoSeries([gpd.geom.Polygon(exterior_coords)], crs="EPSG:5514")[0]
                                )
                            else:
                                # Skip non-polygon geometries
                                log_message(f"Skipping non-polygon geometry: {geom.geom_type}")
                        
                        # Create new GeoDataFrame with transformed geometries
                        gdf_projected = gpd.GeoDataFrame(
                            gdf.drop(columns='geometry'),
                            geometry=transformed_geometries,
                            crs="EPSG:5514"
                        )
                        
                        log_message("Successfully completed manual point-by-point transformation")
                        return gdf_projected, None
                    except Exception as e3:
                        log_message(f"Manual transformation failed: {str(e3)}")
                        
                        # Final fallback: use alternative raster-based approach
                        log_message("All reprojection methods failed. Will attempt direct pixel extraction.")
                        return None, "All reprojection methods failed"
        else:
            # For non-S-JTSK rasters, use the standard approach
            with rasterio.open(raster_path) as src:
                raster_crs = src.crs
            
            # Handle missing CRS in raster
            if raster_crs is None:
                log_message(f"Warning: Raster {os.path.basename(raster_path)} has no CRS defined.")
                return None, "Raster has no CRS"
            
            # Try direct reprojection if CRS differ
            if gdf.crs != raster_crs:
                try:
                    log_message(f"Reprojecting from {gdf.crs} to {raster_crs}")
                    gdf_projected = gdf.to_crs(raster_crs)
                    return gdf_projected, None
                except Exception as e1:
                    log_message(f"Direct reprojection failed: {str(e1)}")
                    
                    # Try with explicit PROJ4 string if available
                    try:
                        log_message("Trying with explicit PROJ4 string...")
                        proj4 = pyproj.CRS(raster_crs).to_proj4()
                        gdf_projected = gdf.to_crs(proj4)
                        return gdf_projected, None
                    except Exception as e2:
                        log_message(f"PROJ4 approach failed: {str(e2)}")
                        
                        # Try two-step reprojection through WGS84
                        try:
                            log_message("Attempting two-step reprojection via EPSG:4326...")
                            gdf_wgs84 = gdf.to_crs(epsg=4326)
                            gdf_projected = gdf_wgs84.to_crs(raster_crs)
                            return gdf_projected, None
                        except Exception as e3:
                            log_message(f"Two-step reprojection failed: {str(e3)}")
                            return None, "All reprojection methods failed"
            else:
                # CRS already match
                return gdf, None
    except Exception as e:
        log_message(f"CRS processing error: {str(e)}")
        return None, f"CRS processing error: {str(e)}"

# Function to extract pixel values from raster for a given shapefile
def extract_values(shapefile_path, raster_path):
    """Extract values from raster that fall within the shapefile polygons using sampling"""
    initial_memory = get_memory_usage()
    log_message(f"Starting extraction. Memory usage: {initial_memory:.1f} MB")
    
    try:
        # Read shapefile
        gdf = gpd.read_file(shapefile_path)
        log_message(f"Processing {os.path.basename(shapefile_path)} with {os.path.basename(raster_path)}")
        
        # Fix CRS issues
        gdf_projected, error = fix_crs_issues(gdf, raster_path)
        
        # If reprojection failed but it's S-JTSK, try alternative approach
        if error and "CHM_uav" in raster_path:
            log_message("Attempting alternative extraction method for S-JTSK data...")
            
            # Since we couldn't reproject, we'll try to extract values using the bounds
            # of the shapefile and then filter post-extraction
            try:
                # Load the raster
                with rasterio.open(raster_path) as src:
                    # Check for NoData value
                    nodata = src.nodata
                    log_message(f"Raster NoData value: {nodata}")
                    
                    # Read a sample of the raster instead of the whole thing
                    # Define a window size based on raster dimensions
                    height, width = src.shape
                    window_height = min(CHUNK_SIZE, height)
                    window_width = min(CHUNK_SIZE, width)
                    
                    # Calculate number of windows needed
                    num_windows_y = (height + window_height - 1) // window_height
                    num_windows_x = (width + window_width - 1) // window_width
                    
                    total_values = []
                    
                    # Track how many samples we've collected
                    sample_count = 0
                    
                    log_message(f"Sampling raster in {num_windows_y}x{num_windows_x} windows...")
                    
                    # Process a subset of windows to save memory
                    # Select a portion of windows to sample randomly
                    max_windows = min(20, num_windows_y * num_windows_x)  # Limit total windows
                    sample_window_indices = np.random.choice(
                        num_windows_y * num_windows_x, 
                        size=max_windows, 
                        replace=False
                    )
                    
                    for window_idx in sample_window_indices:
                        if sample_count >= MAX_SAMPLE_SIZE:
                            break
                            
                        # Convert linear index to 2D coordinates
                        window_y = window_idx // num_windows_x
                        window_x = window_idx % num_windows_x
                        
                        # Calculate window bounds
                        row_start = window_y * window_height
                        col_start = window_x * window_width
                        row_end = min(row_start + window_height, height)
                        col_end = min(col_start + window_width, width)
                        
                        # Filter values properly
                        try:
                            window = src.read(1, window=((row_start, row_end), (col_start, col_end)))
                            
                            # Handle nodata values - first as is
                            if nodata is not None:
                                window = np.where(window == nodata, None, window)
                                
                            # Flatten array
                            flat_values = window.flatten()
                            
                            # Remove None values (if any were substituted)
                            values_list = [v for v in flat_values if v is not None]
                            
                            # Sample if there are too many
                            if len(values_list) > MAX_SAMPLE_SIZE // max_windows:
                                sample_indices = np.random.choice(
                                    len(values_list), 
                                    size=MAX_SAMPLE_SIZE // max_windows, 
                                    replace=False
                                )
                                values_list = [values_list[i] for i in sample_indices]
                            
                            # Add to our collection
                            total_values.extend(values_list)
                            sample_count += len(values_list)
                            
                            log_message(f"  Window {window_idx+1}/{len(sample_window_indices)}: Added {len(values_list)} values")
                            
                        except Exception as e:
                            log_message(f"  Error processing window {window_idx}: {str(e)}")
                            continue
                    
                    # Force garbage collection to free memory
                    gc.collect()
                    
                    log_message(f"Extracted {len(total_values)} sampled values. Memory usage: {get_memory_usage():.1f} MB")
                    return total_values
                    
            except Exception as e:
                log_message(f"Alternative extraction method failed: {str(e)}")
                log_message("Unable to process this raster. Skipping.")
                return []
        
        # If standard reprojection failed and alternative approach isn't applicable
        if error and "CHM_uav" not in raster_path:
            log_message(f"CRS error for {os.path.basename(raster_path)}: {error}")
            return []
            
        # Continue with standard approach if reprojection succeeded
        # Use optimized zonal stats approach with sampling
        try:
            # First check memory usage
            current_memory = get_memory_usage()
            log_message(f"Current memory usage before zonal stats: {current_memory:.1f} MB")
            
            # Use small sample size for each polygon
            all_values = []
            
            # If we have many polygons, sample them
            if len(gdf_projected) > 5:
                # Sample a max of 5 polygons
                sample_size = min(5, len(gdf_projected))
                sample_indices = np.random.choice(len(gdf_projected), size=sample_size, replace=False)
                gdf_sample = gdf_projected.iloc[sample_indices].copy()
                log_message(f"Sampling {sample_size} polygons from {len(gdf_projected)} total")
            else:
                gdf_sample = gdf_projected
            
            # Process each polygon separately to control memory usage
            for idx, row in gdf_sample.iterrows():
                # Create a GeoDataFrame with just this polygon
                single_polygon_gdf = gpd.GeoDataFrame(geometry=[row.geometry], crs=gdf_projected.crs)
                
                # Use zonal_stats with raster_out to get pixel values
                stats = zonal_stats(single_polygon_gdf, raster_path, 
                                   stats=['count'], 
                                   nodata=np.nan,
                                   all_touched=False,
                                   geojson_out=True,
                                   raster_out=True)
                
                # Extract values from the mini raster
                for feature in stats:
                    if 'properties' in feature and 'mini_raster_array' in feature['properties']:
                        mini_raster = feature['properties']['mini_raster_array']
                        
                        # Flatten and filter out NoData values
                        values = mini_raster.flatten()
                        valid_values = values[~np.isnan(values)]
                        
                        # Sample values if there are too many
                        if len(valid_values) > MAX_SAMPLE_SIZE // len(gdf_sample):
                            sample_indices = np.random.choice(
                                len(valid_values),
                                size=MAX_SAMPLE_SIZE // len(gdf_sample),
                                replace=False
                            )
                            valid_values = valid_values[sample_indices]
                        
                        all_values.extend(valid_values.tolist())
                        log_message(f"  Extracted {len(valid_values)} values from polygon {idx}")
                
                # Force garbage collection after each polygon
                gc.collect()
            
            if all_values:
                log_message(f"Successfully extracted {len(all_values)} sampled values. Memory: {get_memory_usage():.1f} MB")
                return all_values
            else:
                log_message("No values extracted with zonal_stats, trying mask method...")
        except Exception as e:
            log_message(f"Zonal stats method failed: {str(e)}")
            log_message("Falling back to mask method...")
        
        # Fallback to mask method with sampling
        log_message("Using mask method with sampling...")
        all_values = []
        
        try:
            with rasterio.open(raster_path) as src:
                # Sample polygons if there are many
                if len(gdf_projected) > 5:
                    sample_size = min(5, len(gdf_projected))
                    sample_indices = np.random.choice(len(gdf_projected), size=sample_size, replace=False)
                    gdf_sample = gdf_projected.iloc[sample_indices].copy()
                else:
                    gdf_sample = gdf_projected
                
                # Process each polygon
                for idx, geom in enumerate(gdf_sample.geometry):
                    try:
                        # Get pixel values within the geometry
                        out_image, out_transform = mask(src, [geom.__geo_interface__], crop=True, nodata=src.nodata)
                        
                        # Handle the output data more carefully
                        try:
                            # Get the nodata value
                            nodata_value = src.nodata
                            
                            # Replace nodata with None
                            if nodata_value is not None:
                                out_image = np.where(out_image == nodata_value, None, out_image)
                            
                            # Get all values
                            flat_values = out_image[0].flatten()
                            
                            # Filter None values
                            values_list = [v for v in flat_values if v is not None]
                            
                            # Sample if there are too many values
                            if len(values_list) > MAX_SAMPLE_SIZE // len(gdf_sample):
                                sample_indices = np.random.choice(
                                    len(values_list),
                                    size=MAX_SAMPLE_SIZE // len(gdf_sample),
                                    replace=False
                                )
                                values_list = [values_list[i] for i in sample_indices]
                            
                            all_values.extend(values_list)
                            log_message(f"  Extracted {len(values_list)} values from geometry {idx+1}/{len(gdf_sample)}")
                            
                        except Exception as e:
                            log_message(f"  Error processing masked data for geometry {idx+1}: {str(e)}")
                            continue
                            
                    except Exception as e:
                        log_message(f"Error masking geometry {idx+1}: {str(e)}")
                        continue
        except Exception as e:
            log_message(f"Mask method failed: {str(e)}")
            
        final_memory = get_memory_usage()
        log_message(f"Total values extracted: {len(all_values)}. Final memory: {final_memory:.1f} MB")
        return all_values
    
    except Exception as e:
        log_message(f"Error processing {os.path.basename(shapefile_path)} with {os.path.basename(raster_path)}: {str(e)}")
        # Force cleanup
        gc.collect()
        return []

# Dictionary to store all values by land cover type
all_values_by_type = {
    "baresoil": [],
    "early_regrowth": [],
    "mature_regrowth": []
}

# Set raster paths that should be skipped (if any)
skip_rasters = []

# Process each shapefile with each raster
for land_type, shapefile_path in shapefiles.items():
    log_message(f"\nProcessing {land_type}...")
    
    # Skip if shapefile doesn't exist
    if not os.path.exists(shapefile_path):
        log_message(f"Warning: {shapefile_path} does not exist. Skipping.")
        continue
    
    # Process each raster file
    for raster_path in tqdm(raster_files, desc=f"Rasters for {land_type}"):
        # Skip problematic rasters if needed
        if os.path.basename(raster_path) in skip_rasters:
            log_message(f"Skipping known problematic raster: {os.path.basename(raster_path)}")
            continue
            
        values = extract_values(shapefile_path, raster_path)
        all_values_by_type[land_type].extend(values)
    
    log_message(f"Collected {len(all_values_by_type[land_type])} pixel values for {land_type}")

# Check if we have values to process
any_values = any(len(values) > 0 for values in all_values_by_type.values())
if not any_values:
    log_message("No valid values were extracted. Check shapefile and raster compatibility.")
    exit(1)

# Configure matplotlib for publication-quality output
import matplotlib as mpl
from matplotlib import rcParams

# Set up publication-quality formatting
rcParams['font.family'] = 'serif'
rcParams['font.serif'] = ['Times New Roman']
rcParams['font.size'] = 10
rcParams['axes.linewidth'] = 0.8
rcParams['axes.labelsize'] = 11
rcParams['axes.titlesize'] = 12
rcParams['xtick.major.width'] = 0.8
rcParams['ytick.major.width'] = 0.8
rcParams['xtick.labelsize'] = 9
rcParams['ytick.labelsize'] = 9
rcParams['legend.fontsize'] = 9
rcParams['figure.dpi'] = 300
rcParams['savefig.dpi'] = 600  # High-res output for publication

# Define publication-quality colors (colorblind-friendly and works in grayscale)
publication_colors = {
    "baresoil": '#E69F00',      # Orange
    "early_regrowth": '#56B4E9', # Blue
    "mature_regrowth": '#009E73' # Green
}

# Update our color scheme
colors = publication_colors

# Create separate publication-quality figures for each land cover type
log_message("\n====== Creating publication-quality separate figures ======")

# Process each land cover type
for land_type, values in all_values_by_type.items():
    log_message(f"\nProcessing figure for {land_type}...")
    
    # Skip if no values
    if not values or len(values) == 0:
        log_message(f"No data for {land_type}, skipping")
        continue
        
    # Clean data for analysis
    clean_values = clean_data_for_analysis(values)
    
    # Check if we have valid data after cleaning
    if len(clean_values) == 0:
        log_message(f"No valid numeric values for {land_type} after cleaning")
        continue
        
    # Additional filtering for extreme values and zeros
    try:
        # Remove infinities and extreme values
        valid_mask = np.isfinite(clean_values) & (np.abs(clean_values) < 1e6) & (np.abs(clean_values) > 1e-6)
        filtered_for_extremes = clean_values[valid_mask]
        
        # Check if we have data left
        if len(filtered_for_extremes) == 0:
            log_message(f"No valid data for {land_type} after filtering extremes and near-zeros")
            
            # Try a less strict filter
            valid_mask = np.isfinite(clean_values)
            filtered_for_extremes = clean_values[valid_mask]
            
            if len(filtered_for_extremes) == 0:
                log_message(f"Still no valid data after basic filtering, skipping {land_type}")
                continue
                
            log_message(f"Using less strict filtering: {len(filtered_for_extremes)} values remain")
            
        # Log statistics of filtered data
        log_message(f"{land_type} filtered data: min={np.min(filtered_for_extremes)}, max={np.max(filtered_for_extremes)}, " + 
                  f"mean={np.mean(filtered_for_extremes)}, median={np.median(filtered_for_extremes)}")
        
        # Further outlier filtering for visualization
        if len(filtered_for_extremes) > 10:
            try:
                # Remove outliers (values beyond 3 standard deviations)
                mean = np.mean(filtered_for_extremes)
                std = np.std(filtered_for_extremes)
                
                if std > 0:  # Only filter if standard deviation is positive
                    filtered_values = filtered_for_extremes[
                        (filtered_for_extremes > mean - 3 * std) & 
                        (filtered_for_extremes < mean + 3 * std)
                    ]
                    
                    # If filtering removed too many values, use original
                    if len(filtered_values) < 0.5 * len(filtered_for_extremes):
                        log_message(f"Warning: Outlier filtering removed too many values. Using pre-filtered values.")
                        filtered_values = filtered_for_extremes
                else:
                    log_message(f"Standard deviation is 0, skipping outlier filtering")
                    filtered_values = filtered_for_extremes
                    
            except Exception as e:
                log_message(f"Error in outlier filtering: {str(e)}")
                filtered_values = filtered_for_extremes
        else:
            filtered_values = filtered_for_extremes
            
    except Exception as e:
        log_message(f"Error filtering extreme values: {str(e)}")
        continue
    
    # Create figure for this land cover type (journal single-column width)
    try:
        # Create a new figure with journal-standard sizing (3.5 inches for single column)
        fig, ax = plt.subplots(figsize=(3.5, 3.5), constrained_layout=True)
        
        # Verify we have valid data to plot
        if len(filtered_values) > 0:
            # Plot histogram with density=True for probability density
            try:
                # Use a reasonable number of bins
                num_bins = min(40, max(10, int(np.sqrt(len(filtered_values)))))
                
                # Plot histogram with improved styling
                n, bins, patches = ax.hist(
                    filtered_values, 
                    bins=num_bins, 
                    alpha=0.7, 
                    color=colors[land_type], 
                    density=False,
                    label=f"n = {len(filtered_values):,}",  # Add comma separators for thousands
                    edgecolor='white',  # White edges for better definition
                    linewidth=0.5       # Thin edges
                )
                
                # Calculate mean safely
                mean_val = np.mean(filtered_values)
                if np.isfinite(mean_val):
                    # Plot vertical line with improved styling
                    ax.axvline(mean_val, color='black', linestyle='--', linewidth=1.2, 
                             label=f'Mean = {mean_val:.2f} m')
                    
                    # Add standard deviation range for publication
                    std_val = np.std(filtered_values)
                    if np.isfinite(std_val) and std_val > 0:
                        # Add shaded area for mean ± 1 std dev
                        ax.axvspan(mean_val - std_val, mean_val + std_val, 
                                 alpha=0.2, color='gray', 
                                 label=f'±1σ = {std_val:.2f} m')
                else:
                    log_message(f"Warning: Mean is not finite ({mean_val})")
                    # Use median instead if mean is problematic
                    median_val = np.median(filtered_values)
                    ax.axvline(median_val, color='black', linestyle='--', linewidth=1.2, 
                             label=f'Median = {median_val:.2f} m')
                
                # Set axis labels with appropriate scale (not scientific notation for reasonable values)
                if np.max(np.abs(filtered_values)) < 1000 and np.max(np.abs(filtered_values)) > 0.01:
                    ax.ticklabel_format(style='plain', axis='x')
                
                # Fix y-axis to start at zero for density plots
                ax.set_ylim(bottom=0)
                
                # Add subtle grid for readability
                ax.grid(True, linestyle='--', alpha=0.3, axis='y')
                
                # Add title and labels with clear formatting
                pretty_name = land_type.replace('_', ' ').title()
                ax.set_title(f'{pretty_name}', fontsize=12, weight='bold')
                ax.set_xlabel('Canopy Height (m)', fontsize=11)
                ax.set_ylabel('Density', fontsize=11)
                
                # Position legend in upper right, with slight transparency
                ax.legend(loc='upper right', framealpha=0.9, edgecolor='lightgray', fontsize=9)
                
            except Exception as e:
                log_message(f"Error creating histogram for {land_type}: {str(e)}")
                # Fallback to basic plot if histogram fails
                ax.plot([0], [0], color=colors[land_type], label=f"Error plotting {land_type}")
        else:
            # No data placeholder
            ax.text(0.5, 0.5, f"No valid data for {land_type}", 
                  horizontalalignment='center', verticalalignment='center',
                  transform=ax.transAxes, fontsize=12, style='italic')
        
        # Save figure in multiple formats for publication
        base_filename = os.path.join(output_folder, f'{land_type}_histogram')
        
        plt.savefig(f"{base_filename}.png", dpi=600, bbox_inches='tight')
        plt.savefig(f"{base_filename}.pdf", format='pdf', bbox_inches='tight')
        plt.savefig(f"{base_filename}.tiff", format='tiff', dpi=600, bbox_inches='tight')
        plt.savefig(f"{base_filename}.eps", format='eps', bbox_inches='tight')
        
        log_message(f"Publication-quality figure for {land_type} saved in multiple formats")
        
        # Close figure to release memory
        plt.close(fig)
        
        # Calculate and save statistics
        try:
            stats = {
                'mean': float(np.mean(filtered_values)) if np.isfinite(np.mean(filtered_values)) else float(np.median(filtered_values)),
                'median': float(np.median(filtered_values)),
                'std_dev': float(np.std(filtered_values)) if np.isfinite(np.std(filtered_values)) else 0.0,
                'min': float(np.min(filtered_values)),
                'max': float(np.max(filtered_values)),
                'count': len(filtered_values),
                '25th_percentile': float(np.percentile(filtered_values, 25)),
                '75th_percentile': float(np.percentile(filtered_values, 75))
            }
            
            # Save statistics to CSV and Excel
            stats_df = pd.DataFrame([stats])
            stats_csv_path = os.path.join(output_folder, f'{land_type}_statistics.csv')
            stats_df.to_csv(stats_csv_path, index=False)
            
            # Also save to Excel with formatting
            excel_path = os.path.join(output_folder, f'{land_type}_statistics.xlsx')
            with pd.ExcelWriter(excel_path) as writer:
                stats_df.to_excel(writer, index=False, sheet_name='Statistics')
                
            log_message(f"Statistics for {land_type} saved to CSV and Excel")
        except Exception as e:
            log_message(f"Error calculating statistics for {land_type}: {str(e)}")
            
    except Exception as e:
        log_message(f"Error creating figure for {land_type}: {str(e)}")
        plt.close()  # Ensure figure is closed in case of error

# Create a summary table with statistics for all land cover types
log_message("\n====== Creating summary statistics table ======")
summary_rows = []

for land_type, values in all_values_by_type.items():
    if not values or len(values) == 0:
        log_message(f"No data for {land_type}, skipping summary statistics")
        continue
    
    # Clean and prepare data
    try:
        # Convert to numeric values and clean the data
        clean_values = clean_data_for_analysis(values)
        
        # Check if we have any valid data after cleaning
        if len(clean_values) == 0:
            log_message(f"No valid numeric data for {land_type} after cleaning for summary stats")
            continue
            
        # Additional filtering for extreme values and zeros
        try:
            # Remove infinities and extreme values
            valid_mask = np.isfinite(clean_values) & (np.abs(clean_values) < 1e6) & (np.abs(clean_values) > 1e-6)
            filtered_values = clean_values[valid_mask]
            
            # Check if we have data left
            if len(filtered_values) == 0:
                log_message(f"No valid data for {land_type} after filtering extremes and near-zeros")
                
                # Try a less strict filter
                valid_mask = np.isfinite(clean_values)
                filtered_values = clean_values[valid_mask]
                
                if len(filtered_values) == 0:
                    log_message(f"Still no valid data after basic filtering, skipping {land_type} summary")
                    continue
                    
                log_message(f"Using less strict filtering for summary: {len(filtered_values)} values remain")
            
            # Calculate statistics safely
            try:
                # Use safer methods for statistics that handle edge cases better
                mean_val = np.mean(filtered_values)
                if not np.isfinite(mean_val):
                    mean_val = np.median(filtered_values)
                    log_message(f"Mean was infinite, using median instead: {mean_val}")
                    
                std_val = np.std(filtered_values)
                if not np.isfinite(std_val):
                    std_val = 0.0
                    log_message(f"Standard deviation was infinite, using 0 instead")
                
                # Create summary row with safer values
                summary_rows.append({
                    'Land_Cover': land_type.replace('_', ' ').title(),
                    'Count': len(filtered_values),
                    'Mean_Height_m': float(mean_val),
                    'Median_Height_m': float(np.median(filtered_values)),
                    'Std_Dev_m': float(std_val),
                    'Min_Height_m': float(np.min(filtered_values)),
                    'Max_Height_m': float(np.max(filtered_values)),
                    'Range_m': float(np.max(filtered_values) - np.min(filtered_values))
                })
                
                log_message(f"Successfully calculated summary statistics for {land_type}")
                
            except Exception as e:
                log_message(f"Error calculating statistics for {land_type}: {str(e)}")
                
        except Exception as e:
            log_message(f"Error filtering data for {land_type} summary: {str(e)}")
            
    except Exception as e:
        log_message(f"Error processing summary for {land_type}: {str(e)}")

if summary_rows:
    summary_df = pd.DataFrame(summary_rows)
    
    # Format the summary dataframe for better readability
    for col in summary_df.columns:
        if col not in ['Land_Cover', 'Count']:
            summary_df[col] = summary_df[col].round(2)
    
    # Save summary table in multiple formats
    summary_csv_path = os.path.join(output_folder, 'canopy_height_summary.csv')
    summary_df.to_csv(summary_csv_path, index=False)
    
    # Create an Excel file with formatting
    summary_excel_path = os.path.join(output_folder, 'canopy_height_summary.xlsx')
    try:
        with pd.ExcelWriter(summary_excel_path) as writer:
            summary_df.to_excel(writer, sheet_name='Summary', index=False)
            
            # Apply formatting to the Excel file
            workbook = writer.book
            worksheet = writer.sheets['Summary']
            
            # Format headers
            header_format = workbook.add_format({
                'bold': True,
                'font_name': 'Arial',
                'font_size': 11,
                'align': 'center',
                'valign': 'vcenter',
                'border': 1
            })
            
            # Format numeric cells
            number_format = workbook.add_format({
                'num_format': '0.00',
                'font_name': 'Arial',
                'font_size': 10,
                'align': 'right'
            })
            
            # Apply formats
            for col_num, col_name in enumerate(summary_df.columns):
                worksheet.write(0, col_num, col_name, header_format)
                
                # Apply number format to numeric columns
                if col_name not in ['Land_Cover', 'Count']:
                    worksheet.set_column(col_num, col_num, 15, number_format)
            
        log_message(f"Summary table saved to {summary_csv_path} and {summary_excel_path}")
    except Exception as e:
        log_message(f"Error saving formatted Excel summary: {str(e)}")
        # Fallback to simple CSV
        summary_df.to_csv(summary_csv_path, index=False)
        log_message(f"Summary table saved to {summary_csv_path} only")
else:
    log_message("No valid data for summary statistics")

log_message("\nAnalysis complete! Results saved to: " + output_folder)

Found 43 raster files

Processing baresoil...


Rasters for baresoil:   0%|                                                                     | 0/43 [00:00<?, ?it/s]

Starting extraction. Memory usage: 247.8 MB
Processing Baresoil.shp with CHM_uav47.tif
Detected S-JTSK Krovak CRS. Manually setting target CRS to EPSG:5514
Successfully reprojected to S-JTSK using EPSG:5514
Current memory usage before zonal stats: 247.8 MB
Sampling 5 polygons from 117 total
  Extracted 20000 values from polygon 40
  Extracted 12852 values from polygon 71
  Extracted 20000 values from polygon 111


Rasters for baresoil:   2%|█▍                                                           | 1/43 [00:00<00:13,  3.17it/s]

  Extracted 20000 values from polygon 103
  Extracted 20000 values from polygon 5
Successfully extracted 92852 sampled values. Memory: 258.6 MB
Starting extraction. Memory usage: 249.4 MB
Processing Baresoil.shp with CHM_uav1.tif
Detected S-JTSK Krovak CRS. Manually setting target CRS to EPSG:5514
Successfully reprojected to S-JTSK using EPSG:5514
Current memory usage before zonal stats: 249.4 MB
Sampling 5 polygons from 117 total
  Extracted 20000 values from polygon 65
  Extracted 20000 values from polygon 34


...

Publication-quality figure for baresoil saved in multiple formats
Statistics for baresoil saved to CSV and Excel

Processing figure for early_regrowth...
Converting object array to numeric (sample: [-1.7976931348623157e+308 -1.7976931348623157e+308
 -1.7976931348623157e+308 None -1.7976931348623157e+308])
Error converting to numeric: 'numpy.ndarray' object has no attribute 'dropna'
Manual conversion yielded 2127297 numeric values
early_regrowth filtered data: min=-0.49920654296875, max=40.6458740234375, mean=0.4914666136878065, median=0.1


The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


Publication-quality figure for early_regrowth saved in multiple formats
Statistics for early_regrowth saved to CSV and Excel

Processing figure for mature_regrowth...
Converting object array to numeric (sample: [-1.7976931348623157e+308 None None -1.7976931348623157e+308 None])
Error converting to numeric: 'numpy.ndarray' object has no attribute 'dropna'
Manual conversion yielded 2132778 numeric values
mature_regrowth filtered data: min=-0.59814453125, max=12.099853515625, mean=3.6621676600447035, median=3.39935302734375


The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


Publication-quality figure for mature_regrowth saved in multiple formats
Statistics for mature_regrowth saved to CSV and Excel

====== Creating summary statistics table ======
Converting object array to numeric (sample: [None -1.7976931348623157e+308 -1.7976931348623157e+308 None
 -1.7976931348623157e+308])
Error converting to numeric: 'numpy.ndarray' object has no attribute 'dropna'
Manual conversion yielded 1931948 numeric values
Successfully calculated summary statistics for baresoil
Converting object array to numeric (sample: [-1.7976931348623157e+308 -1.7976931348623157e+308
 -1.7976931348623157e+308 None -1.7976931348623157e+308])
Error converting to numeric: 'numpy.ndarray' object has no attribute 'dropna'
Manual conversion yielded 2127297 numeric values
Successfully calculated summary statistics for early_regrowth
Converting object array to numeric (sample: [-1.7976931348623157e+308 None None -1.7976931348623157e+308 None])
Error converting to numeric: 'numpy.ndarray' object ha

In [2]:
# Import packages for memory monitoring
import os
import gc
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import pyproj
from rasterio.mask import mask
import matplotlib.pyplot as plt
from rasterstats import zonal_stats
import glob
from tqdm import tqdm
import warnings
import psutil  # For memory monitoring

# Suppress common warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Memory monitor function
def get_memory_usage():
    """Return the current memory usage in MB"""
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss / 1024 / 1024  # Convert to MB

# Configuration parameters
MAX_SAMPLE_SIZE = 100000  # Maximum number of pixels to sample per shapefile and raster
MEMORY_THRESHOLD_MB = 1000  # Trigger memory optimization if usage exceeds this value
CHUNK_SIZE = 520  # Size of chunks when reading large rasters

# Set folders
data_folder = "H:/classification"  # Replace with your actual folder path
shapefile_folder = os.path.join(data_folder, "shapefiles")
raster_folder = os.path.join(data_folder, "rasters")
output_folder = os.path.join(data_folder, "results")

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Define shapefiles and raster patterns
shapefiles = {
    "baresoil": os.path.join(shapefile_folder, "Baresoil.shp"),
    "early_regrowth": os.path.join(shapefile_folder, "Early_regrowth.shp"),
    "mature_regrowth": os.path.join(shapefile_folder, "Mature_regrowth2.shp")
}

# Find all raster files - using CHM pattern based on error message
raster_files = glob.glob(os.path.join(raster_folder, "CHM_uav*.tif"))
print(f"Found {len(raster_files)} raster files")

# Create a log file for errors
log_file = os.path.join(output_folder, "processing_log.txt")
with open(log_file, 'w') as f:
    f.write("Canopy Height Processing Log\n")
    f.write("--------------------------\n")

def log_message(message):
    """Write message to log file and print to console"""
    print(message)
    with open(log_file, 'a') as f:
        f.write(message + "\n")

# Function to clean and validate data for analysis
def clean_data_for_analysis(values):
    """Convert values to numeric and filter invalid entries"""
    if not values or len(values) == 0:
        return np.array([])
        
    # First, convert list to numpy array if it's not already
    if not isinstance(values, np.ndarray):
        values_array = np.array(values, dtype=object)
    else:
        values_array = values
    
    # Check if we need to convert from object type to numeric
    if values_array.dtype.kind == 'O':  # Object type
        log_message(f"Converting object array to numeric (sample: {values_array[:5]})")
        
        # Filter out None values first
        values_filtered = [v for v in values_array if v is not None]
        
        if len(values_filtered) == 0:
            log_message("All values were None, returning empty array")
            return np.array([])
            
        # Try to convert to float
        try:
            # Use pandas for better mixed-type conversion
            import pandas as pd
            numeric_values = pd.to_numeric(values_filtered, errors='coerce').dropna().values
            log_message(f"Converted {len(numeric_values)} values to numeric (out of {len(values_filtered)})")
            
            if len(numeric_values) == 0:
                log_message("No valid numeric values after conversion")
                return np.array([])
                
            return numeric_values
        except Exception as e:
            log_message(f"Error converting to numeric: {str(e)}")
            
            # Manual conversion as fallback
            numeric_values = []
            for v in values_filtered:
                try:
                    if v is not None:
                        numeric_values.append(float(v))
                except (ValueError, TypeError):
                    continue
                    
            log_message(f"Manual conversion yielded {len(numeric_values)} numeric values")
            return np.array(numeric_values)
    else:
        # Already numeric type, just filter NaN and inf values if float
        if values_array.dtype.kind in 'fc':  # float or complex
            mask = ~np.isnan(values_array) & np.isfinite(values_array)
            return values_array[mask]
        else:
            # For integers, just return as is
            return values_array

# Function to verify and fix CRS issues
def fix_crs_issues(gdf, raster_path):
    """Try multiple approaches to fix CRS issues between shapefile and raster"""
    try:
        # Handle missing CRS in shapefile
        if gdf.crs is None:
            log_message(f"Shapefile has no CRS defined. Setting to EPSG:4326 (WGS84)")
            gdf = gdf.set_crs(epsg=4326)
        
        # Check if raster path contains "S-JTSK" or "Krovak" in its CRS
        if "CHM_uav" in raster_path:
            log_message(f"Detected S-JTSK Krovak CRS. Manually setting target CRS to EPSG:5514")
            
            try:
                target_crs = "EPSG:5514"
                gdf_projected = gdf.to_crs(target_crs)
                log_message("Successfully reprojected to S-JTSK using EPSG:5514")
                return gdf_projected, None
            except Exception as e1:
                log_message(f"EPSG:5514 reprojection failed: {str(e1)}")
                
                try:
                    s_jtsk_wkt = """
                    PROJCS["S-JTSK / Krovak East North",
                        GEOGCS["S-JTSK",
                            DATUM["System_of_the_Unified_Trigonometrical_Cadastral_Network",
                                SPHEROID["Bessel 1841",6377397.155,299.1528128,
                                    AUTHORITY["EPSG","7004"]],
                                TOWGS84[589,76,480,0,0,0,0],
                                AUTHORITY["EPSG","6156"]],
                            PRIMEM["Greenwich",0,
                                AUTHORITY["EPSG","8901"]],
                            UNIT["degree",0.0174532925199433,
                                AUTHORITY["EPSG","9122"]],
                            AUTHORITY["EPSG","4156"]],
                        PROJECTION["Krovak"],
                        PARAMETER["latitude_of_center",49.5],
                        PARAMETER["longitude_of_center",24.83333333333333],
                        PARAMETER["azimuth",30.28813972222222],
                        PARAMETER["pseudo_standard_parallel_1",78.5],
                        PARAMETER["scale_factor",0.9999],
                        PARAMETER["false_easting",0],
                        PARAMETER["false_northing",0],
                        UNIT["metre",1,
                            AUTHORITY["EPSG","9001"]],
                        AXIS["X",EAST],
                        AXIS["Y",NORTH],
                        AUTHORITY["EPSG","5514"]]
                    """
                    gdf_projected = gdf.to_crs(s_jtsk_wkt)
                    log_message("Successfully reprojected to S-JTSK using WKT definition")
                    return gdf_projected, None
                except Exception as e2:
                    log_message(f"WKT reprojection failed: {str(e2)}")
                    
                    try:
                        log_message("Attempting direct coordinate transformation...")
                        transformer = pyproj.Transformer.from_pipeline(
                            "+proj=pipeline +step +proj=unitconvert +xy_in=deg +xy_out=rad "
                            "+step +proj=krovak +lat_0=49.5 +lon_0=24.83333333333333 "
                            "+alpha=30.28813972222222 +k=0.9999 +x_0=0 +y_0=0"
                        )
                        
                        transformed_geometries = []
                        for geom in gdf.geometry:
                            if geom.geom_type == 'Polygon':
                                exterior_coords = []
                                for x, y in geom.exterior.coords:
                                    e, n = transformer.transform(y, x)
                                    exterior_coords.append((e, n))
                                transformed_geometries.append(
                                    gpd.GeoSeries([gpd.geom.Polygon(exterior_coords)], crs="EPSG:5514")[0]
                                )
                            else:
                                log_message(f"Skipping non-polygon geometry: {geom.geom_type}")
                        
                        gdf_projected = gpd.GeoDataFrame(
                            gdf.drop(columns='geometry'),
                            geometry=transformed_geometries,
                            crs="EPSG:5514"
                        )
                        
                        log_message("Successfully completed manual point-by-point transformation")
                        return gdf_projected, None
                    except Exception as e3:
                        log_message(f"Manual transformation failed: {str(e3)}")
                        log_message("All reprojection methods failed. Will attempt direct pixel extraction.")
                        return None, "All reprojection methods failed"
        else:
            with rasterio.open(raster_path) as src:
                raster_crs = src.crs
            
            if raster_crs is None:
                log_message(f"Warning: Raster {os.path.basename(raster_path)} has no CRS defined.")
                return None, "Raster has no CRS"
            
            if gdf.crs != raster_crs:
                try:
                    log_message(f"Reprojecting from {gdf.crs} to {raster_crs}")
                    gdf_projected = gdf.to_crs(raster_crs)
                    return gdf_projected, None
                except Exception as e1:
                    log_message(f"Direct reprojection failed: {str(e1)}")
                    
                    try:
                        log_message("Trying with explicit PROJ4 string...")
                        proj4 = pyproj.CRS(raster_crs).to_proj4()
                        gdf_projected = gdf.to_crs(proj4)
                        return gdf_projected, None
                    except Exception as e2:
                        log_message(f"PROJ4 approach failed: {str(e2)}")
                        
                        try:
                            log_message("Attempting two-step reprojection via EPSG:4326...")
                            gdf_wgs84 = gdf.to_crs(epsg=4326)
                            gdf_projected = gdf_wgs84.to_crs(raster_crs)
                            return gdf_projected, None
                        except Exception as e3:
                            log_message(f"Two-step reprojection failed: {str(e3)}")
                            return None, "All reprojection methods failed"
            else:
                return gdf, None
    except Exception as e:
        log_message(f"CRS processing error: {str(e)}")
        return None, f"CRS processing error: {str(e)}"

# Function to extract pixel values from raster for a given shapefile
def extract_values(shapefile_path, raster_path):
    """Extract values from raster that fall within the shapefile polygons using sampling"""
    initial_memory = get_memory_usage()
    log_message(f"Starting extraction. Memory usage: {initial_memory:.1f} MB")
    
    try:
        gdf = gpd.read_file(shapefile_path)
        log_message(f"Processing {os.path.basename(shapefile_path)} with {os.path.basename(raster_path)}")
        
        gdf_projected, error = fix_crs_issues(gdf, raster_path)
        
        if error and "CHM_uav" in raster_path:
            log_message("Attempting alternative extraction method for S-JTSK data...")
            
            try:
                with rasterio.open(raster_path) as src:
                    nodata = src.nodata
                    log_message(f"Raster NoData value: {nodata}")
                    
                    height, width = src.shape
                    window_height = min(CHUNK_SIZE, height)
                    window_width = min(CHUNK_SIZE, width)
                    
                    num_windows_y = (height + window_height - 1) // window_height
                    num_windows_x = (width + window_width - 1) // window_width
                    
                    total_values = []
                    sample_count = 0
                    
                    log_message(f"Sampling raster in {num_windows_y}x{num_windows_x} windows...")
                    
                    max_windows = min(20, num_windows_y * num_windows_x)
                    sample_window_indices = np.random.choice(
                        num_windows_y * num_windows_x, 
                        size=max_windows, 
                        replace=False
                    )
                    
                    for window_idx in sample_window_indices:
                        if sample_count >= MAX_SAMPLE_SIZE:
                            break
                            
                        window_y = window_idx // num_windows_x
                        window_x = window_idx % num_windows_x
                        
                        row_start = window_y * window_height
                        col_start = window_x * window_width
                        row_end = min(row_start + window_height, height)
                        col_end = min(col_start + window_width, width)
                        
                        try:
                            window = src.read(1, window=((row_start, row_end), (col_start, col_end)))
                            
                            if nodata is not None:
                                window = np.where(window == nodata, None, window)
                                
                            flat_values = window.flatten()
                            values_list = [v for v in flat_values if v is not None]
                            
                            if len(values_list) > MAX_SAMPLE_SIZE // max_windows:
                                sample_indices = np.random.choice(
                                    len(values_list), 
                                    size=MAX_SAMPLE_SIZE // max_windows, 
                                    replace=False
                                )
                                values_list = [values_list[i] for i in sample_indices]
                            
                            total_values.extend(values_list)
                            sample_count += len(values_list)
                            
                            log_message(f"  Window {window_idx+1}/{len(sample_window_indices)}: Added {len(values_list)} values")
                            
                        except Exception as e:
                            log_message(f"  Error processing window {window_idx}: {str(e)}")
                            continue
                    
                    gc.collect()
                    
                    log_message(f"Extracted {len(total_values)} sampled values. Memory usage: {get_memory_usage():.1f} MB")
                    return total_values
                    
            except Exception as e:
                log_message(f"Alternative extraction method failed: {str(e)}")
                log_message("Unable to process this raster. Skipping.")
                return []
        
        if error and "CHM_uav" not in raster_path:
            log_message(f"CRS error for {os.path.basename(raster_path)}: {error}")
            return []
            
        try:
            current_memory = get_memory_usage()
            log_message(f"Current memory usage before zonal stats: {current_memory:.1f} MB")
            
            all_values = []
            
            if len(gdf_projected) > 5:
                sample_size = min(5, len(gdf_projected))
                sample_indices = np.random.choice(len(gdf_projected), size=sample_size, replace=False)
                gdf_sample = gdf_projected.iloc[sample_indices].copy()
                log_message(f"Sampling {sample_size} polygons from {len(gdf_projected)} total")
            else:
                gdf_sample = gdf_projected
            
            for idx, row in gdf_sample.iterrows():
                single_polygon_gdf = gpd.GeoDataFrame(geometry=[row.geometry], crs=gdf_projected.crs)
                
                stats = zonal_stats(single_polygon_gdf, raster_path, 
                                   stats=['count'], 
                                   nodata=np.nan,
                                   all_touched=False,
                                   geojson_out=True,
                                   raster_out=True)
                
                for feature in stats:
                    if 'properties' in feature and 'mini_raster_array' in feature['properties']:
                        mini_raster = feature['properties']['mini_raster_array']
                        values = mini_raster.flatten()
                        valid_values = values[~np.isnan(values)]
                        
                        if len(valid_values) > MAX_SAMPLE_SIZE // len(gdf_sample):
                            sample_indices = np.random.choice(
                                len(valid_values),
                                size=MAX_SAMPLE_SIZE // len(gdf_sample),
                                replace=False
                            )
                            valid_values = valid_values[sample_indices]
                        
                        all_values.extend(valid_values.tolist())
                        log_message(f"  Extracted {len(valid_values)} values from polygon {idx}")
                
                gc.collect()
            
            if all_values:
                log_message(f"Successfully extracted {len(all_values)} sampled values. Memory: {get_memory_usage():.1f} MB")
                return all_values
            else:
                log_message("No values extracted with zonal_stats, trying mask method...")
        except Exception as e:
            log_message(f"Zonal stats method failed: {str(e)}")
            log_message("Falling back to mask method...")
        
        log_message("Using mask method with sampling...")
        all_values = []
        
        try:
            with rasterio.open(raster_path) as src:
                if len(gdf_projected) > 5:
                    sample_size = min(5, len(gdf_projected))
                    sample_indices = np.random.choice(len(gdf_projected), size=sample_size, replace=False)
                    gdf_sample = gdf_projected.iloc[sample_indices].copy()
                else:
                    gdf_sample = gdf_projected
                
                for idx, geom in enumerate(gdf_sample.geometry):
                    try:
                        out_image, out_transform = mask(src, [geom.__geo_interface__], crop=True, nodata=src.nodata)
                        nodata_value = src.nodata
                        if nodata_value is not None:
                            out_image = np.where(out_image == nodata_value, None, out_image)
                        flat_values = out_image[0].flatten()
                        values_list = [v for v in flat_values if v is not None]
                        if len(values_list) > MAX_SAMPLE_SIZE // len(gdf_sample):
                            sample_indices = np.random.choice(
                                len(values_list),
                                size=MAX_SAMPLE_SIZE // len(gdf_sample),
                                replace=False
                            )
                            values_list = [values_list[i] for i in sample_indices]
                        all_values.extend(values_list)
                        log_message(f"  Extracted {len(values_list)} values from geometry {idx+1}/{len(gdf_sample)}")
                    except Exception as e:
                        log_message(f"Error masking geometry {idx+1}: {str(e)}")
                        continue
        except Exception as e:
            log_message(f"Mask method failed: {str(e)}")
            
        final_memory = get_memory_usage()
        log_message(f"Total values extracted: {len(all_values)}. Final memory: {final_memory:.1f} MB")
        return all_values
    
    except Exception as e:
        log_message(f"Error processing {os.path.basename(shapefile_path)} with {os.path.basename(raster_path)}: {str(e)}")
        gc.collect()
        return []

# Dictionary to store all values by land cover type
all_values_by_type = {
    "baresoil": [],
    "early_regrowth": [],
    "mature_regrowth": []
}

# Set raster paths that should be skipped (if any)
skip_rasters = []

# Process each shapefile with each raster
for land_type, shapefile_path in shapefiles.items():
    log_message(f"\nProcessing {land_type}...")
    
    if not os.path.exists(shapefile_path):
        log_message(f"Warning: {shapefile_path} does not exist. Skipping.")
        continue
    
    for raster_path in tqdm(raster_files, desc=f"Rasters for {land_type}"):
        if os.path.basename(raster_path) in skip_rasters:
            log_message(f"Skipping known problematic raster: {os.path.basename(raster_path)}")
            continue
        values = extract_values(shapefile_path, raster_path)
        all_values_by_type[land_type].extend(values)
    
    log_message(f"Collected {len(all_values_by_type[land_type])} pixel values for {land_type}")

# Check if we have values to process
any_values = any(len(values) > 0 for values in all_values_by_type.values())
if not any_values:
    log_message("No valid values were extracted. Check shapefile and raster compatibility.")
    exit(1)

# Configure matplotlib for publication-quality output
import matplotlib as mpl
from matplotlib import rcParams

rcParams['font.family'] = 'serif'
rcParams['font.serif'] = ['Times New Roman']
rcParams['font.size'] = 10
rcParams['axes.linewidth'] = 0.8
rcParams['axes.labelsize'] = 11
rcParams['axes.titlesize'] = 12
rcParams['xtick.major.width'] = 0.8
rcParams['ytick.major.width'] = 0.8
rcParams['xtick.labelsize'] = 9
rcParams['ytick.labelsize'] = 9
rcParams['legend.fontsize'] = 9
rcParams['figure.dpi'] = 300
rcParams['savefig.dpi'] = 600

publication_colors = {
    "baresoil": '#E69F00',
    "early_regrowth": '#56B4E9',
    "mature_regrowth": '#009E73'
}

colors = publication_colors

# Create separate publication-quality figures for each land cover type
log_message("\n====== Creating publication-quality separate figures ======")

for land_type, values in all_values_by_type.items():
    log_message(f"\nProcessing figure for {land_type}...")
    
    if not values or len(values) == 0:
        log_message(f"No data for {land_type}, skipping")
        continue
        
    # Clean data for analysis
    clean_values = clean_data_for_analysis(values)
    # Filter out negative values (since canopy heights should be non-negative)
    clean_values = clean_values[clean_values >= 0]
    
    # Check if we have valid data after cleaning
    if len(clean_values) == 0:
        log_message(f"No valid numeric values for {land_type} after cleaning")
        continue
        
    # Additional filtering for extreme values
    try:
        valid_mask = np.isfinite(clean_values) & (clean_values < 1e6)
        filtered_for_extremes = clean_values[valid_mask]
        
        if len(filtered_for_extremes) == 0:
            log_message(f"No valid data for {land_type} after filtering extremes")
            continue
                
        log_message(f"{land_type} filtered data: min={np.min(filtered_for_extremes)}, max={np.max(filtered_for_extremes)}, " + 
                  f"mean={np.mean(filtered_for_extremes)}, median={np.median(filtered_for_extremes)}")
        
        # Further outlier filtering for visualization
        if len(filtered_for_extremes) > 10:
            try:
                mean = np.mean(filtered_for_extremes)
                std = np.std(filtered_for_extremes)
                
                if std > 0:
                    filtered_values = filtered_for_extremes[
                        (filtered_for_extremes > mean - 3 * std) & 
                        (filtered_for_extremes < mean + 3 * std)
                    ]
                    
                    if len(filtered_values) < 0.5 * len(filtered_for_extremes):
                        log_message(f"Warning: Outlier filtering removed too many values. Using pre-filtered values.")
                        filtered_values = filtered_for_extremes
                else:
                    log_message(f"Standard deviation is 0, skipping outlier filtering")
                    filtered_values = filtered_for_extremes
            except Exception as e:
                log_message(f"Error in outlier filtering: {str(e)}")
                filtered_values = filtered_for_extremes
        else:
            filtered_values = filtered_for_extremes
            
    except Exception as e:
        log_message(f"Error filtering extreme values: {str(e)}")
        continue
    
    # Create figure for this land cover type
    try:
        fig, ax = plt.subplots(figsize=(3.5, 3.5), constrained_layout=True)
        
        if len(filtered_values) > 0:
            num_bins = min(40, max(10, int(np.sqrt(len(filtered_values)))))
            
            n, bins, patches = ax.hist(
                filtered_values, 
                bins=num_bins, 
                alpha=0.7, 
                color=colors[land_type], 
                density=True,
                label=f"n = {len(filtered_values):,}",
                edgecolor='white',
                linewidth=0.5
            )
            
            # Verify the area under the histogram
            area = sum(n * (bins[1:] - bins[:-1]))
            log_message(f"Area under histogram for {land_type}: {area:.3f}")
            
            mean_val = np.mean(filtered_values)
            if np.isfinite(mean_val):
                ax.axvline(mean_val, color='black', linestyle='--', linewidth=1.2, 
                         label=f'Mean = {mean_val:.2f} m')
                
                std_val = np.std(filtered_values)
                if np.isfinite(std_val) and std_val > 0:
                    ax.axvspan(mean_val - std_val, mean_val + std_val, 
                             alpha=0.2, color='gray', 
                             label=f'±1σ = {std_val:.2f} m')
            else:
                median_val = np.median(filtered_values)
                ax.axvline(median_val, color='black', linestyle='--', linewidth=1.2, 
                         label=f'Median = {median_val:.2f} m')
            
            if np.max(np.abs(filtered_values)) < 1000 and np.max(np.abs(filtered_values)) > 0.01:
                ax.ticklabel_format(style='plain', axis='x')
            
            ax.set_ylim(bottom=0)
            ax.set_xlim(left=0)  # Force x-axis to start at 0
            ax.grid(True, linestyle='--', alpha=0.3, axis='y')
            
            pretty_name = land_type.replace('_', ' ').title()
            ax.set_title(f'{pretty_name}', fontsize=12, weight='bold')
            ax.set_xlabel('Canopy Height (m)', fontsize=11)
            ax.set_ylabel('Density', fontsize=11)
            ax.legend(loc='upper right', framealpha=0.9, edgecolor='lightgray', fontsize=9)
        else:
            ax.text(0.5, 0.5, f"No valid data for {land_type}", 
                  horizontalalignment='center', verticalalignment='center',
                  transform=ax.transAxes, fontsize=12, style='italic')
        
        base_filename = os.path.join(output_folder, f'{land_type}_histogram')
        plt.savefig(f"{base_filename}.png", dpi=600, bbox_inches='tight')
        plt.savefig(f"{base_filename}.pdf", format='pdf', bbox_inches='tight')
        plt.savefig(f"{base_filename}.tiff", format='tiff', dpi=600, bbox_inches='tight')
        plt.savefig(f"{base_filename}.eps", format='eps', bbox_inches='tight')
        
        log_message(f"Publication-quality figure for {land_type} saved in multiple formats")
        plt.close(fig)
        
        # Calculate and save statistics
        try:
            stats = {
                'mean': float(np.mean(filtered_values)) if np.isfinite(np.mean(filtered_values)) else float(np.median(filtered_values)),
                'median': float(np.median(filtered_values)),
                'std_dev': float(np.std(filtered_values)) if np.isfinite(np.std(filtered_values)) else 0.0,
                'min': float(np.min(filtered_values)),
                'max': float(np.max(filtered_values)),
                'count': len(filtered_values),
                '25th_percentile': float(np.percentile(filtered_values, 25)),
                '75th_percentile': float(np.percentile(filtered_values, 75))
            }
            
            stats_df = pd.DataFrame([stats])
            stats_csv_path = os.path.join(output_folder, f'{land_type}_statistics.csv')
            stats_df.to_csv(stats_csv_path, index=False)
            
            excel_path = os.path.join(output_folder, f'{land_type}_statistics.xlsx')
            with pd.ExcelWriter(excel_path) as writer:
                stats_df.to_excel(writer, index=False, sheet_name='Statistics')
                
            log_message(f"Statistics for {land_type} saved to CSV and Excel")
        except Exception as e:
            log_message(f"Error calculating statistics for {land_type}: {str(e)}")
            
    except Exception as e:
        log_message(f"Error creating figure for {land_type}: {str(e)}")
        plt.close()

# Create a summary table with statistics for all land cover types
log_message("\n====== Creating summary statistics table ======")
summary_rows = []

for land_type, values in all_values_by_type.items():
    if not values or len(values) == 0:
        log_message(f"No data for {land_type}, skipping summary statistics")
        continue
    
    try:
        clean_values = clean_data_for_analysis(values)
        clean_values = clean_values[clean_values >= 0]
        
        if len(clean_values) == 0:
            log_message(f"No valid numeric data for {land_type} after cleaning for summary stats")
            continue
            
        try:
            valid_mask = np.isfinite(clean_values) & (clean_values < 1e6)
            filtered_values = clean_values[valid_mask]
            
            if len(filtered_values) == 0:
                log_message(f"No valid data for {land_type} after filtering extremes")
                continue
                
            try:
                mean_val = np.mean(filtered_values)
                if not np.isfinite(mean_val):
                    mean_val = np.median(filtered_values)
                    log_message(f"Mean was infinite, using median instead: {mean_val}")
                    
                std_val = np.std(filtered_values)
                if not np.isfinite(std_val):
                    std_val = 0.0
                    log_message(f"Standard deviation was infinite, using 0 instead")
                
                summary_rows.append({
                    'Land_Cover': land_type.replace('_', ' ').title(),
                    'Count': len(filtered_values),
                    'Mean_Height_m': float(mean_val),
                    'Median_Height_m': float(np.median(filtered_values)),
                    'Std_Dev_m': float(std_val),
                    'Min_Height_m': float(np.min(filtered_values)),
                    'Max_Height_m': float(np.max(filtered_values)),
                    'Range_m': float(np.max(filtered_values) - np.min(filtered_values))
                })
                
                log_message(f"Successfully calculated summary statistics for {land_type}")
                
            except Exception as e:
                log_message(f"Error calculating statistics for {land_type}: {str(e)}")
                
        except Exception as e:
            log_message(f"Error filtering data for {land_type} summary: {str(e)}")
            
    except Exception as e:
        log_message(f"Error processing summary for {land_type}: {str(e)}")

if summary_rows:
    summary_df = pd.DataFrame(summary_rows)
    
    for col in summary_df.columns:
        if col not in ['Land_Cover', 'Count']:
            summary_df[col] = summary_df[col].round(2)
    
    summary_csv_path = os.path.join(output_folder, 'canopy_height_summary.csv')
    summary_df.to_csv(summary_csv_path, index=False)
    
    summary_excel_path = os.path.join(output_folder, 'canopy_height_summary.xlsx')
    try:
        with pd.ExcelWriter(summary_excel_path) as writer:
            summary_df.to_excel(writer, sheet_name='Summary', index=False)
            
            workbook = writer.book
            worksheet = writer.sheets['Summary']
            
            header_format = workbook.add_format({
                'bold': True,
                'font_name': 'Arial',
                'font_size': 11,
                'align': 'center',
                'valign': 'vcenter',
                'border': 1
            })
            
            number_format = workbook.add_format({
                'num_format': '0.00',
                'font_name': 'Arial',
                'font_size': 10,
                'align': 'right'
            })
            
            for col_num, col_name in enumerate(summary_df.columns):
                worksheet.write(0, col_num, col_name, header_format)
                if col_name not in ['Land_Cover', 'Count']:
                    worksheet.set_column(col_num, col_num, 15, number_format)
            
        log_message(f"Summary table saved to {summary_csv_path} and {summary_excel_path}")
    except Exception as e:
        log_message(f"Error saving formatted Excel summary: {str(e)}")
        summary_df.to_csv(summary_csv_path, index=False)
        log_message(f"Summary table saved to {summary_csv_path} only")
else:
    log_message("No valid data for summary statistics")

log_message("\nAnalysis complete! Results saved to: " + output_folder)

Found 43 raster files

Processing baresoil...


Rasters for baresoil:   0%|                                                                     | 0/43 [00:00<?, ?it/s]

Starting extraction. Memory usage: 150.0 MB
Processing Baresoil.shp with CHM_uav47.tif
Detected S-JTSK Krovak CRS. Manually setting target CRS to EPSG:5514
Successfully reprojected to S-JTSK using EPSG:5514
Current memory usage before zonal stats: 164.9 MB
Sampling 5 polygons from 117 total
  Extracted 20000 values from polygon 9
  Extracted 20000 values from polygon 85
  Extracted 20000 values from polygon 39
  Extracted 20000 values from polygon 14


Rasters for baresoil:   2%|█▍                                                           | 1/43 [00:00<00:23,  1.78it/s]

  Extracted 15386 values from polygon 74
Successfully extracted 95386 sampled values. Memory: 184.5 MB
Starting extraction. Memory usage: 180.1 MB
Processing Baresoil.shp with CHM_uav1.tif
Detected S-JTSK Krovak CRS. Manually setting target CRS to EPSG:5514
Successfully reprojected to S-JTSK using EPSG:5514
Current memory usage before zonal stats: 180.2 MB
Sampling 5 polygons from 117 total
  Extracted 5250 values from polygon 72
  Extracted 20000 values from polygon 48
  Extracted 20000 values from polygon 26


...

Publication-quality figure for baresoil saved in multiple formats
Statistics for baresoil saved to CSV and Excel

Processing figure for early_regrowth...
Converting object array to numeric (sample: [None -1.7976931348623157e+308 None -1.7976931348623157e+308 None])
Error converting to numeric: 'numpy.ndarray' object has no attribute 'dropna'
Manual conversion yielded 2147077 numeric values
early_regrowth filtered data: min=6.103515625e-05, max=30.94403076171875, mean=0.6167198856477651, median=0.42926025390625
Area under histogram for early_regrowth: 1.000


The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


Publication-quality figure for early_regrowth saved in multiple formats
Statistics for early_regrowth saved to CSV and Excel

Processing figure for mature_regrowth...
Converting object array to numeric (sample: [None -1.7976931348623157e+308 -1.7976931348623157e+308 None None])
Error converting to numeric: 'numpy.ndarray' object has no attribute 'dropna'
Manual conversion yielded 2024100 numeric values
mature_regrowth filtered data: min=0.000244140625, max=11.31036376953125, mean=3.4003614483639226, median=3.578521728515625
Area under histogram for mature_regrowth: 1.000


The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


Publication-quality figure for mature_regrowth saved in multiple formats
Statistics for mature_regrowth saved to CSV and Excel

====== Creating summary statistics table ======
Converting object array to numeric (sample: [None -1.7976931348623157e+308 -1.7976931348623157e+308 None
 -1.7976931348623157e+308])
Error converting to numeric: 'numpy.ndarray' object has no attribute 'dropna'
Manual conversion yielded 1939688 numeric values
Successfully calculated summary statistics for baresoil
Converting object array to numeric (sample: [None -1.7976931348623157e+308 None -1.7976931348623157e+308 None])
Error converting to numeric: 'numpy.ndarray' object has no attribute 'dropna'
Manual conversion yielded 2147077 numeric values
Successfully calculated summary statistics for early_regrowth
Converting object array to numeric (sample: [None -1.7976931348623157e+308 -1.7976931348623157e+308 None None])
Error converting to numeric: 'numpy.ndarray' object has no attribute 'dropna'
Manual conversion